In [1]:
from jellyfish import damerau_levenshtein_distance

In [2]:
file_name = "dict.txt"
with open(file_name, encoding="utf-8") as file_in:
    dictionary = file_in.read().splitlines()
dictionary[:10]

['абажур',
 'абажурчик',
 'абазин',
 'абазинец',
 'абазинцы',
 'абазинка',
 'абазия',
 'абандон',
 'аббат',
 'аббатиса']

In [3]:
len(dictionary)

62027

In [4]:
file_name = "queries.txt"
with open(file_name, encoding="utf-8") as file_in:
    queries = file_in.read().splitlines()
queries[:10]

['воспитаннясть',
 'неоплатоэивк',
 'махаьный',
 'втникж',
 'эйдетки',
 'лепидосьрне',
 'эътомолог',
 'патуинча',
 'утожишоки',
 'дружеественнось']

In [5]:
len(queries)

20000

In [6]:
def generate_intermediate_words(w: str) -> set[str]:
    words = set()
    for i in range(len(w)):
        for j in range(ord("а"), ord("я") + 1):
            words.add(w[:i] + chr(j) + w[i + 1:])
    for i in range(len(w) + 1):
        for j in range(ord("а"), ord("я") + 1):
            words.add(w[:i] + chr(j) + w[i:])
    for i in range(len(w)):
        words.add(w[:i] + w[i + 1:])
    for i in range(len(w) - 1):
        words.add(w[:i] + w[i + 1] + w[i] + w[i + 2:])
    return words

In [7]:
def find_intermediate_word(w1: str, w2: str) -> str:
    words = generate_intermediate_words(w1)
    for word in words:
        if damerau_levenshtein_distance(word, w2) == 1:
            return word

In [8]:
def correct_typo(w1: str) -> str:
    wd1 = wd2 = ""
    for w2 in dictionary:
        d = damerau_levenshtein_distance(w1, w2)
        if d == 0:
            return f"{w1} 0"
        elif d == 1 and not wd1:
            wd1 = w2
        elif d == 2 and not wd2:
            wd2 = w2
    if wd1:
        return f"{w1} 1 {wd1}"
    if wd2:
        wd1 = find_intermediate_word(w1, wd2)
        return f"{w1} 2 {wd1} {wd2}"
    return f"{w1} 3+"

In [9]:
for query in ["пончик", "поник", "пончек", "пночикк", "поньчиг", "пннчнкк"]:
    print(correct_typo(query))

пончик 0
поник 1 пончик
пончек 1 пончик
пночикк 2 пночик пончик
поньчиг 2 пончиг пончик
пннчнкк 3+


In [10]:
corrections = []
for query in queries[:10]:
    corrections.append(correct_typo(query))
corrections[:10]

['воспитаннясть 1 воспитанность',
 'неоплатоэивк 2 неоплатоэик неоплатоник',
 'махаьный 1 махальный',
 'втникж 2 втник ватник',
 'эйдетки 1 эйдетик',
 'лепидосьрне 2 лепидосирне лепидосирен',
 'эътомолог 1 энтомолог',
 'патуинча 2 патуина патина',
 'утожишоки 3+',
 'дружеественнось 2 дружественнось дружественность']

In [ ]:
file_name = "corrections.txt"
with open(file_name, "w", encoding="utf-8") as file_out:
    file_out.write("\n".join(corrections))